# Neural Network Decoding

We will decode the current choice of the animal given information from the past 10 trials. In a previous step, we reduced the number of features to the 30 most relevant ones. We will now train a neural network on these reduced inputs. First, we import relevant modules and define useful functions. 

In [100]:
ROOT = '/Users/pablomartin/python'
folder = ROOT + '/DATA_structures/featureSelection/'
original = ROOT + '/DATA_structures/'

import pickle
import os
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split, cross_val_score

def predict_NN(NN, X, Y):
    Y_prediction = NN.predict(X)
    Y_prediction = (Y_prediction > 0.5)
    return float(np.sum(Y_prediction.ravel() == Y))/len(Y)

def train_NN(X,Y, hidden_layers, BS, EPS = 50):

    #Initializing Neural Network
    classifier = Sequential()
    
    # Adding input layer 
    classifier.add(Dense(units = hidden_layers['1'],
                         kernel_initializer = 'uniform',
                         activation = 'relu',
                         input_dim = X.shape[1]))

    # Adding hidden layer
    for layer in hidden_layers.keys():
        if hidden_layers[layer] > 0:
            classifier.add(Dense(units = hidden_layers[layer],
                                 kernel_initializer = 'uniform',
                                 activation = 'relu'))

    # Adding the output layer
    classifier.add(Dense(units = 1,
                         kernel_initializer = 'uniform',
                         activation = 'sigmoid'))
    
    # Compiling Neural Network
    classifier.compile(optimizer = 'adam',
                       loss = 'binary_crossentropy',
                       metrics = ['accuracy'])

    classifier.fit(X, Y, batch_size = BS, epochs = EPS, verbose=0)
    return classifier






OK, let's give it a go. Randomly picking the number of units in the hidden layer to be 20, and the training batch size to be 20, we will see how a perceptron decodes the choices of rats. We will divide the data into training and testing sets as usual, EXCEPT, that we will separate it by leaving 2 sessions aside as the testing set and train on the remainding data. We want to be able to see where in the session the neural networks are not decoding properly. If we divide using the usual train_test_split section, we will get tons of holes in our sessions. Most datasets have ~11 sessions, so leaving 2 sessions out is equivalent to leaving 20% of the data out. 

In [83]:
import random
import time

idx = pd.IndexSlice


#for starters
unitNo = {'1': 30, '2': 5}
batchSize = 100
    
feature_labels = [str(X.columns.levels[0][a]) + '_' +  str(X.columns.levels[1][b]) \
             for a,b in zip(X.columns.labels[0], X.columns.labels[1])]   
        

networks = []
datasets = []
for fileName in os.listdir(folder):


    #loading reduced data
    X,y = pickle.load(open(folder + '/' + fileName, 'rb'))
    
    noSessions = range(len(X.index.levels[0]))
    random.shuffle(noSessions)
    
    #we must pick 2 sessions randomly  
    train_sessions = idx[['S' + str(w) for w in noSessions[2:]], :, :]
    test_sessions = idx[['S' + str(w) for w in noSessions[:2]], :, :]
    
    training_dataset = [X.loc[train_sessions, :], y.loc[train_sessions]]
    testing_dataset = [X.loc[test_sessions, :], y.loc[test_sessions]]
    data_splits = [training_dataset, testing_dataset]
    #want to save each datasplit
    datasets.append(data_splits)
    
    
    start = time.time()
    print 'training network...%s' %(fileName)
    NN = train_NN(training_dataset[0], training_dataset[1], unitNo, batchSize)
    print 'finished training. elapsed time: %1.2f' %((time.time() - start) / 60)
    
    #we want to save each NN to play around with later
    networks.append(NN)
    #quick score
    print 'score:%1.3f' %predict_NN(NN, testing_dataset[0], testing_dataset[1])

training network...PSR_TbyT_Saline.p
finished training. elapsed time: 0.50
score:0.685
training network...PSR_TbyT_FirstTraining.p
finished training. elapsed time: 0.20
score:0.682
training network...PSR_TbyT_MPFC.p
finished training. elapsed time: 0.26
score:0.706
training network...PSR_TbyT_Ipsi.p
finished training. elapsed time: 0.27
score:0.588
training network...DSR_TbyT_Saline.p
finished training. elapsed time: 0.51
score:0.780
training network...DSR_TbyT_Ipsi.p
finished training. elapsed time: 0.30
score:0.700
training network...PSR_TbyT_Contra.p
finished training. elapsed time: 0.41
score:0.546
training network...DSR_TbyT_FirstTraining.p
finished training. elapsed time: 0.23
score:0.629
training network...DSR_TbyT_MPFC.p
finished training. elapsed time: 0.29
score:0.711
training network...PSR_TbyT_OFC.p
finished training. elapsed time: 0.30
score:0.599
training network...PSR_TbyT_MidTraining.p
finished training. elapsed time: 0.37
score:0.596
training network...DSR_TbyT_MidTrai

Before we go and optimize the neural network, we want to get a sense of how it's decoding trials. We want to build the tools to visualize this anyways. We want a function that given a session, it graphs where the reversals were and where the decoder made a mistake. 

In [77]:
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter


def plot_decoder_accuracy(y, decoder, title = 'Learning Curve'):
    noTrials = y.shape[0]
    moving_window = 5
    moving_average = np.zeros(noTrials - moving_window, dtype=float)
    for trial in range(noTrials - moving_window):
        moving_average[trial] = np.nanmean(y.iloc[trial : trial + moving_window])
        
    #applying Savitzky-Golay filter to smooth curve
    moving_average = savgol_filter(moving_average, moving_window, 2)
    
    blocks = y.groupby(axis = 0, level = 'block')
    revPoints = [len(b) for label, b in blocks]
    revPoints = np.cumsum(revPoints)
    
    
    fig, ax = plt.subplots(figsize=(12,5))
    
    #plotting learning curve
    ax.plot(range(noTrials - moving_window), moving_average)
    
    #plotting reversal block points
    for xc in revPoints:
        plt.axvline(x = xc, color='green')
        
    #what trials did neural network miscode
    trials_decoded = (decoder == y)    
    misDecoded = np.nonzero(trials_decoded == 0)[0]
    #plot as red dot on top of learning curve
    ax.scatter(misDecoded, [0.5] * len(misDecoded), marker='o', c='red')
    
    
    ax.set_ylabel('P(choosing W)', FontSize=12)
    ax.set_xlabel('trial #', FontSize=12)
    ax.set_title(title, FontSize = 14)
    plt.show()
    
    


In [ ]:

    
for datasetIndex, fileName in enumerate(os.listdir(folder)):   

    training_dataset, testing_dataset = datasets[datasetIndex]    

    #session data
    iterator = testing_dataset[0].groupby(axis=0, level='session')
    #current choice
    y_ = testing_dataset[1].groupby(axis=0, level='session')

    currentNN = networks[datasetIndex]

    for session, y in zip(iterator, y_):
        #running neural network on session
        decoder = (currentNN.predict(session[1]) > 0.5).ravel()
        print 'score for session: %1.3f' %(float(np.sum(decoder == y[1])) / len(y[1]))

        #creating label for session
        title = fileName + '_' + y[0]
        #plotting session
        plot_decoder_accuracy(y[1], decoder, title = title)


Now we will optimize each neural network. Since they are very slow to train, we will only explore a limited feature space. We will start by separating each dataset into three sets: training, validating, and testing. We will use the training set to (duh) train the neural network and to test it on the validation set. A final unbiased test will be performed on the testing set. 

The feature space will consists of varying three variables:  
1) number of units in hidden layers  
2) whether there is 1 or 2 hidden layers  
3) batch size    

Unfortunately, since neural networks are probabilistic, we will need to train 3 times per condition and average result. We will try the following parameters:  

1) [5, 10, 30, 100] # of units  
2) [1, 2] # of hidden layers
3) [10, 50, 100] batch size    

We are looking at 4x2x3x3 = 72 networks to train per dataset. It takes ~ 2 minutes to trian each network, so we are looking at 33 hours processing time total without parallelization. Accross 4 cores it *should* be around 8 hours. We will try to do this overnight. 


In [ ]:
from sklearn.model_selection import train_test_split

    

    
#let's split the data into three sets but with testing set being 2 entire sessions
fileNames = os.listdir(folder)
for fileName in fileNames[3:]:

    scores = np.full([20, 3], np.NaN)

    #loading reduced data
    X,y = pickle.load(open(folder + '/' + fileName, 'rb'))
    
    noSessions = range(len(X.index.levels[0]))
    random.shuffle(noSessions)
    
    #we must pick 2 sessions randomly  
    train_sessions = idx[['S' + str(w) for w in noSessions[2:]], :, :]
    test_sessions = idx[['S' + str(w) for w in noSessions[:2]], :, :]
    
    training_dataset = [X.loc[train_sessions, :], y.loc[train_sessions]]
    testing_dataset = [X.loc[test_sessions, :], y.loc[test_sessions]]
    
    
    
    X_train, X_validate, y_train, y_validate = train_test_split(training_dataset[0],
                                                                training_dataset[1],
                                                                test_size=0.20,
                                                                random_state = 23)

    data_splits = [training_dataset, testing_dataset]
    print 'dataset: %s\n' %fileName
    print '# of training examples: %i' %X_train.shape[0]
    print '# of validation set examples: %i' %X_validate.shape[0]
    print '# of examples in testing set: %i\n\n' %testing_dataset[0].shape[0]
    
    
    
    
    units = [5, 10, 30, 100]
    layer1 = [w for s in range(5) for w in units]
    layer2 = [w for w in [-1] + units for s in range(4)]
    batchSizes = [10, 50, 100]
    for rIndex, (units1, units2) in enumerate(zip(layer1, layer2)):
        #unit structure
        unitNo = {'1': units1, '2' : units2}
        for batchIndex, batchSize in enumerate(batchSizes):
            
            #train neural network
            start = time.time()
            
            tmp_score = []
            for shuffle in range(3):
                NN = train_NN(X_train, y_train, unitNo, batchSize)
                
                #calculate score on validation set
                tmp_score.append(predict_NN(NN, X_validate, y_validate))
                #tmp_score.append(np.random.random())
            
            print 'NN %i/%i finished - processing time: %1.2f minutes - score:%1.3f' \
                 %(1 + rIndex * 3 + batchIndex, 60, ((time.time() - start) / 60), np.nanmean(tmp_score))    
            scores[rIndex, batchIndex] = np.nanmean(tmp_score)
    
    print scores
    max_condition = np.nonzero(scores == np.max(scores))
    model_params = (layer1[max_condition[0][0]], layer2[max_condition[0][0]], batchSizes[max_condition[1][0]])
    print '\n\n'
    print '*' * 80
    print 'max score: %1.3f' %np.max(scores)
    print 'units layer 1: %i\nunits layer 2: %i\nbatch size: %i' %model_params
    print '*' * 80
    print '\n\n'

dataset: PSR_TbyT_Ipsi.p

# of training examples: 780
# of validation set examples: 195
# of examples in testing set: 292


NN 1/60 finished - processing time: 15.04 minutes - score:0.638
NN 2/60 finished - processing time: 4.51 minutes - score:0.634
NN 3/60 finished - processing time: 3.72 minutes - score:0.629
NN 4/60 finished - processing time: 8.24 minutes - score:0.593
NN 5/60 finished - processing time: 4.39 minutes - score:0.621
NN 6/60 finished - processing time: 5.05 minutes - score:0.631
NN 7/60 finished - processing time: 7.78 minutes - score:0.574
NN 8/60 finished - processing time: 4.39 minutes - score:0.573
NN 9/60 finished - processing time: 3.70 minutes - score:0.609
NN 10/60 finished - processing time: 7.79 minutes - score:0.556
NN 11/60 finished - processing time: 4.20 minutes - score:0.528
NN 12/60 finished - processing time: 3.54 minutes - score:0.540
NN 13/60 finished - processing time: 8.33 minutes - score:0.574
NN 14/60 finished - processing time: 4.28 minutes - 